In [1]:
import anndata as ad
import pandas as pd
import scanpy as sc
from datasets import Dataset
import pyarrow as pa
from collections import defaultdict
import numpy as np
import pickle
import pyreadr

In [5]:
gd_path = '/home/comp/cszrwang/data/osmfish/osmfish.cell_proportion.txt'
gd = pd.read_csv(gd_path, index_col=0, header=0,delimiter='\t')
gd.sum(axis=1).max()

20

In [2]:
st_path = '/home/comp/cszrwang/data/osmfish/osmfish.st.cnt.genexrow.tsv'
st = pd.read_csv(st_path, index_col=0, header=0,delimiter='\t')
st

,400_27600,400_28400,400_29200,400_30000,400_30800,400_31600,400_32400,400_33200,400_34000,400_34800,...,22000_35600,22000_38000,22000_39600,22800_31600,22800_33200,22800_34000,22800_34800,22800_35600,22800_37200,23600_37200
Gad2,31,10,1,17,2,19,183,14,44,239,...,149,15,12,12,10,19,53,5,13,25
Slc32a1,3,6,0,3,2,8,84,4,3,125,...,5,25,5,0,0,0,13,1,1,2
Crhbp,2,8,0,2,0,1,79,3,0,11,...,6,28,21,2,5,1,0,0,5,0
Cnr1,1,14,1,10,0,16,1,9,38,48,...,15,23,17,6,28,16,67,6,9,4
Vip,5,9,1,10,1,28,33,19,34,17,...,17,43,9,4,6,8,81,3,11,13
Cpne5,0,19,1,0,0,0,24,9,6,18,...,1,0,0,2,4,2,8,0,0,0
Pthlh,0,11,0,0,0,0,5,4,6,10,...,5,25,5,1,1,2,26,2,3,2
Crh,0,10,0,0,0,2,17,4,8,26,...,7,19,15,3,6,3,53,5,2,0
Tbr1,50,19,2,42,20,38,23,88,220,216,...,41,14,0,138,73,26,61,8,31,28
Lamp5,2,31,0,2,3,4,3,26,114,321,...,158,19,15,172,233,149,84,18,3,6


In [3]:
st_adata = ad.AnnData(
    X=st.T.values,
    obs=pd.DataFrame(index=st.columns), # Creating a default index for observations
    var=pd.DataFrame(index=st.index)  # Creating default index for variables
)
st_adata

AnnData object with n_obs × n_vars = 737 × 33

In [4]:
sc_path = '/home/comp/cszrwang/data/osmfish/SSp_ref/external/sc_cnt.33gene.5392cell.genexrow.tsv'
scc = pd.read_csv(sc_path, index_col=0, header=0,delimiter='\t')
scc

,F2S4_160722_001_A01,F2S4_160722_001_B01,F2S4_160722_001_C01,F2S4_160722_001_D01,F2S4_160722_001_E01,F2S4_160722_001_H01,F2S4_160722_002_A01,F2S4_160722_002_B01,F2S4_160722_002_C01,F2S4_160722_002_D01,...,F2S4_181129_011_G01,F2S4_181129_011_H01,F2S4_181129_012_A01,F2S4_181129_012_B01,F2S4_181129_012_C01,F2S4_181129_012_D01,F2S4_181129_012_E01,F2S4_181129_012_F01,F2S4_181129_012_G01,F2S4_181129_012_H01
Gad2,4809,2591,0,0,768,3032,3046,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Slc32a1,104,85,0,0,90,62,20,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Crhbp,0,1,0,0,0,4169,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cnr1,2413,31,1,0,453,391,1151,1,453,288,...,67,161,29,10,0,87,5,75,0,24
Vip,13,27158,3,2,6,1,40613,1,0,2,...,0,0,0,0,0,0,0,0,0,0
Cpne5,107,0,1,28,57,1,0,4,77,62,...,10,91,107,58,50,26,2,60,5,21
Pthlh,0,1708,0,0,0,0,997,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Crh,4353,0,0,0,1,291,2650,1,0,0,...,0,28,0,18,0,0,3,6,0,0
Tbr1,0,0,84,203,0,0,0,119,114,120,...,29,102,33,90,83,46,50,57,5,31
Lamp5,0,0,0,282,0,0,0,1,1908,0,...,52,653,1393,1529,184,215,2766,2002,29,1667


In [5]:
sc_adata = ad.AnnData(
    X=scc.T.values,
    obs=pd.DataFrame(index=scc.columns), # Creating a default index for observations
    var=pd.DataFrame(index=scc.index)  # Creating default index for variables
)
sc_adata

AnnData object with n_obs × n_vars = 5392 × 33

In [6]:
sc_meta_path = '/home/comp/cszrwang/data/osmfish/SSp_ref/external/sc_mta.5392cell.tsv'
sc_meta = pd.read_csv(sc_meta_path, index_col=0, header=0,delimiter='\t')
sc_meta

,bio_celltype
sample_name,
F2S4_160816_020_C01,Lamp5
F2S4_170215_029_F01,Lamp5
F2S4_170215_029_H01,Lamp5
F2S4_170215_026_H01,Lamp5
F2S4_170216_031_G01,Lamp5
...,...
F2S4_160824_041_D01,L6 IT CTX
F2S4_160824_041_E01,L6 CT CTX
F2S4_160824_041_F01,L6 IT CTX


In [7]:
sc_adata.obs['celltype'] = sc_meta['bio_celltype']

In [8]:
sc.pp.filter_genes(sc_adata, min_cells=1)
sc.pp.filter_cells(sc_adata, min_genes=1)
sc_adata

AnnData object with n_obs × n_vars = 5392 × 33
    obs: 'celltype', 'n_genes'
    var: 'n_cells'

In [9]:
sc_genes = sc_adata.var.index.values
st_genes = st_adata.var.index.values
common_genes = set(sc_genes).intersection(set(st_genes))
sc_adata = sc_adata[:,list(common_genes)]
st_adata = st_adata[:,list(common_genes)]

In [11]:
sc_adata.write('/home/comp/cszrwang/data/osmfish/sc_osm.h5ad')

In [12]:
sc_adata.obs['celltype'].value_counts()

celltype
L4/5 IT CTX      1385
L2/3 IT CTX-1     962
L6 IT CTX         792
L6b CTX           618
L6 CT CTX         514
Vip               275
L5 IT CTX         232
L5 NP CTX         222
Lamp5             185
Sst               160
L5 PT CTX          47
Name: count, dtype: int64

In [19]:
# Convert the index to a column
st_adata.obs['index'] = st_adata.obs.index

# Apply the function to split the index column and create new columns
st_adata.obs['x'] = st_adata.obs['index'].apply(lambda x: float(x.split('_')[0])/ 800)
st_adata.obs['y'] = st_adata.obs['index'].apply(lambda x: float(x.split('_')[1])/ 800)

In [20]:
st_adata.obs

,index,x,y
400_27600,400_27600,0.5,34.5
400_28400,400_28400,0.5,35.5
400_29200,400_29200,0.5,36.5
400_30000,400_30000,0.5,37.5
400_30800,400_30800,0.5,38.5
...,...,...,...
22800_34000,22800_34000,28.5,42.5
22800_34800,22800_34800,28.5,43.5
22800_35600,22800_35600,28.5,44.5
22800_37200,22800_37200,28.5,46.5


In [21]:
spatial_cor = pd.DataFrame(st_adata.obs[['x','y']])
spatial_cor

,x,y
400_27600,0.5,34.5
400_28400,0.5,35.5
400_29200,0.5,36.5
400_30000,0.5,37.5
400_30800,0.5,38.5
...,...,...
22800_34000,28.5,42.5
22800_34800,28.5,43.5
22800_35600,28.5,44.5
22800_37200,28.5,46.5


In [22]:
spatial_cor.to_csv("/home/comp/cszrwang/data/osmfish/osm_cor.txt", sep='\t', index=False)

In [18]:
st_adata.write('/home/comp/cszrwang/data/osmfish/st_osm.h5ad')